In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [2]:
tittles=[]
years=[]
certificates=[]
runtimes=[]
genres=[]
ratings=[]
metascore=[]
overview = []
directors=[]
actors=[]
listFilm=[]
count =0

In [3]:
for i in range (0,1500, 51):
  url  = "https://www.imdb.com/search/title/?title_type=feature&year=1945-01-01,2021-12-31&sort=num_votes,desc&start="+str(i)+"&ref_=adv_nxt"
  response = requests.get(url, headers={"Accept-Language": "en-US"})
  soup = BeautifulSoup(response.text, "html.parser")
  count=count+1
  movies= soup.find_all('div', class_='lister-item mode-advanced')

  for movie in movies:
    aFilm=[]
    tittle = movie.h3.find('a', href=True).text
    year = movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
    year = year.split(" ")
    if len(year)>1:
      year=year[1]
      #tittle=tittle+year[0]
      #print(tittle,year)
    else:
      year=year[0]
    aFilm.append(tittle)
    aFilm.append(year)

    rating=float(movie.strong.text)
    aFilm.append(rating)

    nv = movie.find_all('span', attrs = {'name':'nv'})
    vote=nv[0].text
    aFilm.append(vote)
    
    meta = movie.find("div", class_="ratings-metascore")
    if meta is not None:
      metascore = meta.span.get_text()
    else:
      meta = "Unknown"
    aFilm.append(metascore)
    
    gross= nv[1].text if len(nv) > 1 else '-'
    aFilm.append(gross) 
  
    ct=movie.find_all('span', class_="certificate")
    certificate=ct[0].text
    aFilm.append(certificate)

    rt=movie.find_all('span', class_="runtime")
    runtime=rt[0].text
    aFilm.append(runtime)

    gr=movie.find_all('span', class_="genre")
    genre=gr[0].text
    genre=genre[1:]
    genre=genre[:-12]
    aFilm.append(genre)
    
    overview = movie.find_all("p", class_="text-muted")[1].text
    aFilm.append(overview)
    
    star=[]
    filminfo1=movie.find_all('p',class_='')
    f2=filminfo1[0]
    director=f2.find_all('a')[0]
    director = director.text
    
    for a in f2.find_all('a')[1:]:
      star.append(a.text)
  
    aFilm.append(star)

    if len(star) >4:
      director=star[0]+"; "+director
      del star[0]
    aFilm.append(director)
    listFilm.append(aFilm)

In [4]:
df = pd.DataFrame(listFilm, columns =['Title', 'Year', 'Rating','Vote','Metascore','Budget','Certificate','Runtime','Genre','Overview','Artist','Director'])
df.head()

,Title,Year,Rating,Vote,Metascore,Budget,Certificate,Runtime,Genre,Overview,Artist,Director
0,The Shawshank Redemption,(1994),9.3,"2,751,116",82,$28.34M,R,142 min,Drama,"\nOver the course of several years, two convic...","[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Frank Darabont
1,The Dark Knight,(2008),9.0,"2,723,845",84,$534.86M,PG-13,152 min,"Action, Crime, Drama",\nWhen the menace known as the Joker wreaks ha...,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",Christopher Nolan
2,Inception,(2010),8.8,"2,417,677",74,$292.58M,PG-13,148 min,"Action, Adventure, Sci-Fi",\nA thief who steals corporate secrets through...,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",Christopher Nolan
3,Fight Club,(1999),8.8,"2,190,708",66,$37.03M,R,139 min,Drama,\nAn insomniac office worker and a devil-may-c...,"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",David Fincher
4,Forrest Gump,(1994),8.8,"2,140,219",82,$330.25M,PG-13,142 min,"Drama, Romance","\nThe presidencies of Kennedy and Johnson, the...","[Tom Hanks, Robin Wright, Gary Sinise, Sally F...",Robert Zemeckis


In [5]:
df.shape

(1500, 12)

In [6]:
df.to_csv("Movies.csv")

In [7]:
df=pd.read_csv("Movies.csv")

In [8]:
df.isna().sum()

Unnamed: 0     0
Title          0
Year           0
Rating         0
Vote           0
Metascore      0
Budget         0
Certificate    0
Runtime        0
Genre          0
Overview       0
Artist         0
Director       0
dtype: int64

In [9]:
df.shape

(1500, 13)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1500 non-null   int64  
 1   Title        1500 non-null   object 
 2   Year         1500 non-null   object 
 3   Rating       1500 non-null   float64
 4   Vote         1500 non-null   object 
 5   Metascore    1500 non-null   int64  
 6   Budget       1500 non-null   object 
 7   Certificate  1500 non-null   object 
 8   Runtime      1500 non-null   object 
 9   Genre        1500 non-null   object 
 10  Overview     1500 non-null   object 
 11  Artist       1500 non-null   object 
 12  Director     1500 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 152.5+ KB


In [11]:
df.describe()

,Unnamed: 0,Rating,Metascore
count,1500.000000,1500.000000,1500.000000
mean,749.500000,7.172867,64.379333
std,433.157015,0.781834,16.171635
min,0.000000,1.900000,16.000000
25%,374.750000,6.600000,52.000000
50%,749.500000,7.200000,65.000000
75%,1124.250000,7.700000,76.000000
max,1499.000000,9.300000,100.000000


In [12]:
df.iloc[:59]

,Unnamed: 0,Title,Year,Rating,Vote,Metascore,Budget,Certificate,Runtime,Genre,Overview,Artist,Director
0,0,The Shawshank Redemption,(1994),9.3,"2,751,116",82,$28.34M,R,142 min,Drama,"\nOver the course of several years, two convic...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",Frank Darabont
1,1,The Dark Knight,(2008),9.0,"2,723,845",84,$534.86M,PG-13,152 min,"Action, Crime, Drama",\nWhen the menace known as the Joker wreaks ha...,"['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",Christopher Nolan
2,2,Inception,(2010),8.8,"2,417,677",74,$292.58M,PG-13,148 min,"Action, Adventure, Sci-Fi",\nA thief who steals corporate secrets through...,"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ...",Christopher Nolan
3,3,Fight Club,(1999),8.8,"2,190,708",66,$37.03M,R,139 min,Drama,\nAn insomniac office worker and a devil-may-c...,"['Brad Pitt', 'Edward Norton', 'Meat Loaf', 'Z...",David Fincher
4,4,Forrest Gump,(1994),8.8,"2,140,219",82,$330.25M,PG-13,142 min,"Drama, Romance","\nThe presidencies of Kennedy and Johnson, the...","['Tom Hanks', 'Robin Wright', 'Gary Sinise', '...",Robert Zemeckis
5,5,Pulp Fiction,(1994),8.9,"2,113,442",95,$107.93M,R,154 min,"Crime, Drama","\nThe lives of two mob hitmen, a boxer, a gang...","['John Travolta', 'Uma Thurman', 'Samuel L. Ja...",Quentin Tarantino
6,6,The Matrix,(1999),8.7,"1,961,283",73,$171.48M,R,136 min,"Action, Sci-Fi",\nWhen a beautiful stranger leads computer hac...,"['Keanu Reeves', 'Laurence Fishburne', 'Carrie...",Lilly Wachowski; Lana Wachowski
7,7,The Lord of the Rings: The Fellowship of the Ring,(2001),8.8,"1,918,805",92,$315.54M,PG-13,178 min,"Action, Adventure, Drama",\nA meek Hobbit from the Shire and eight compa...,"['Elijah Wood', 'Ian McKellen', 'Orlando Bloom...",Peter Jackson
8,8,Interstellar,(2014),8.7,"1,916,598",74,$188.02M,PG-13,169 min,"Adventure, Drama, Sci-Fi",\nWhen Earth becomes uninhabitable in the futu...,"['Matthew McConaughey', 'Anne Hathaway', 'Jess...",Christopher Nolan
9,9,The Godfather,(1972),9.2,"1,913,676",100,$134.97M,R,175 min,"Crime, Drama","\nDon Vito Corleone, head of a mafia family, d...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",Francis Ford Coppola
